### Importing Necessary Libraries

In [31]:
import pandas as pd
import plotly.graph_objects as pgo
from nsepy import get_history
from datetime import date
from dateutil.relativedelta import relativedelta

### Functions history_func and ema_calc

In [32]:
# Returns history of NSE data as a dataframe for specified number of years before current date
def history_func(number_of_years):
    current_day = date.today()
    start_day = current_day - relativedelta(years = number_of_years)

    history_df = get_history(symbol = 'NIFTY 50', start = start_day, end = current_day, index=True).reset_index()

    return(history_df)

In [33]:
# Returns a list of the exponential moving average in a financial dataframe for a specific number of days
def ema_calc(dataframe, number_of_days):
    sma = 0
    for j in range(number_of_days):
                sma += dataframe[j]
    sma /= number_of_days

    multiplier = 2 / (number_of_days + 1)
    ema = []
    for i in dataframe.index:
        if(i < number_of_days):
            ema.append(round(sma, 2))
        else: 
            ema_value = (dataframe[i] * multiplier) + (ema[i-1] * (1 - multiplier))
            ema.append(round(ema_value, 2))
    
    return(ema)

### 15 years of historical data

In [34]:
# Call the get history function for 15 years
df_nse_15_years = history_func(15)
df_nse_15_years = df_nse_15_years[["Date", "Open", "High", "Low", "Close"]]
df_nse_15_years

,Date,Open,High,Low,Close
0,2008-01-29,5279.55,5391.60,5225.25,5280.80
1,2008-01-30,5283.75,5314.30,5142.25,5167.60
2,2008-01-31,5172.25,5251.65,5071.15,5137.45
3,2008-02-01,5140.60,5339.95,5090.75,5317.25
4,2008-02-04,5315.55,5545.20,5315.55,5463.50
...,...,...,...,...,...
3709,2023-01-20,18115.60,18145.45,18016.20,18027.65
3710,2023-01-23,18118.45,18162.60,18063.45,18118.55
3711,2023-01-24,18183.95,18201.25,18078.65,18118.30
3712,2023-01-25,18093.35,18100.60,17846.15,17891.95


### List of Gap Up/Gap Down Dates for 5 years of Data

In [35]:
# Call the get history function for 5 years
df_nse_analysis_5_years = history_func(5)
df_nse_analysis_5_years = df_nse_analysis_5_years.drop(['Volume', 'Turnover'], axis=1)

gap_up_list = []
gap_down_list = []

# Add date to list if there is a difference of 200 in their respective directions
for i in df_nse_analysis_5_years.index:
    if(i == 0):
        continue

    diff = df_nse_analysis_5_years["Close"][i-1] - df_nse_analysis_5_years["Open"][i]
    if(diff > 200):
        gap_down_list.append(str(df_nse_analysis_5_years["Date"][i]))
    elif(diff < -200):
        gap_up_list.append(str(df_nse_analysis_5_years["Date"][i]))

print("Dates with Gap up of 200 or more:", gap_up_list)
print()
print("Dates with Gap down of 200 or more:", gap_down_list)
print()
print("Number of days where there was a gap up in the last 5 years:", len(gap_up_list))
print("Number of days where there was a gap down in the last 5 years:", len(gap_down_list))

Dates with Gap up of 200 or more: ['2019-05-20', '2019-09-23', '2020-03-24', '2020-03-27', '2020-03-31', '2020-04-07', '2020-04-09', '2020-04-15', '2020-04-17', '2020-04-30', '2020-05-13', '2020-06-16', '2022-02-25', '2022-03-10', '2022-03-16', '2022-03-17', '2022-05-20', '2022-06-27', '2022-07-20', '2022-10-04', '2022-10-14', '2022-11-11']

Dates with Gap down of 200 or more: ['2018-02-06', '2018-10-11', '2020-02-28', '2020-03-06', '2020-03-09', '2020-03-12', '2020-03-13', '2020-03-16', '2020-03-19', '2020-03-23', '2020-03-30', '2020-04-21', '2020-05-04', '2020-06-12', '2021-02-26', '2021-03-04', '2021-04-19', '2022-01-27', '2022-02-14', '2022-02-22', '2022-02-24', '2022-03-02', '2022-03-07', '2022-04-18', '2022-05-06', '2022-05-19', '2022-06-13', '2022-08-29', '2022-09-01', '2022-09-14', '2022-10-10']

Number of days where there was a gap up in the last 5 years: 22
Number of days where there was a gap down in the last 5 years: 31


### More analysis of 5 years of data

In [36]:
# Add column (High-Low) to analysis dataframe
df_nse_analysis_5_years['(High-Low)'] = round(df_nse_analysis_5_years['High'] - df_nse_analysis_5_years['Low'], 2)


max_movement = []

# Add max movement in positive or negative direction into list
for i in df_nse_analysis_5_years.index:
    high_movement = df_nse_analysis_5_years['High'][i] - df_nse_analysis_5_years['Open'][i]
    low_movement = df_nse_analysis_5_years['Low'][i] - df_nse_analysis_5_years['Open'][i]
    if(abs(low_movement) > high_movement):
        max_movement.append(round(low_movement, 2))
    else:
        max_movement.append(round(high_movement, 2))

# Add list as a new column to analysis dataframe
df_nse_analysis_5_years['Movement from Open'] = max_movement
del max_movement

pct_change = []

# Add percentage change of close from previous close into list 
for i in df_nse_analysis_5_years.index:
    if(i == 0):
        pct_change.append(float("NaN"))
        continue

    pct_change.append(round(((df_nse_analysis_5_years['Close'][i] - df_nse_analysis_5_years['Close'][i-1]) / df_nse_analysis_5_years['Close'][i-1] * 100), 2))

# Add list as a new column to analysis dataframe
df_nse_analysis_5_years['%Change Prev Close'] = pct_change
del pct_change

# Add 7, 14, 21, 50, 200 day exponential moving averages to dataframe
ema_list = ema_calc(df_nse_analysis_5_years['Close'], 7)
df_nse_analysis_5_years['7 day EMA'] = ema_list

ema_list = ema_calc(df_nse_analysis_5_years['Close'], 14)
df_nse_analysis_5_years['14 day EMA'] = ema_list

ema_list = ema_calc(df_nse_analysis_5_years['Close'], 21)
df_nse_analysis_5_years['21 day EMA'] = ema_list

ema_list = ema_calc(df_nse_analysis_5_years['Close'], 50)
df_nse_analysis_5_years['50 day EMA'] = ema_list

ema_list = ema_calc(df_nse_analysis_5_years['Close'], 200)
df_nse_analysis_5_years['200 day EMA'] = ema_list

# Save this dataframe as a csv file
df_nse_analysis_5_years.to_csv('NSE 5 year data and analysis.csv', index=False)
df_nse_analysis_5_years

,Date,Open,High,Low,Close,(High-Low),Movement from Open,%Change Prev Close,7 day EMA,14 day EMA,21 day EMA,50 day EMA,200 day EMA
0,2018-01-29,11079.35,11171.55,11075.95,11130.40,95.60,92.20,NaN,10878.58,10692.64,10611.61,10413.12,10750.59
1,2018-01-30,11120.85,11121.10,11033.90,11049.65,87.20,-86.95,-0.73,10878.58,10692.64,10611.61,10413.12,10750.59
2,2018-01-31,11018.80,11058.50,10979.30,11027.70,79.20,39.70,-0.20,10878.58,10692.64,10611.61,10413.12,10750.59
3,2018-02-01,11044.55,11117.35,10878.80,11016.90,238.55,-165.75,-0.10,10878.58,10692.64,10611.61,10413.12,10750.59
4,2018-02-02,10938.20,10954.95,10736.10,10760.60,218.85,-202.10,-2.33,10878.58,10692.64,10611.61,10413.12,10750.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,2023-01-20,18115.60,18145.45,18016.20,18027.65,129.25,-99.40,-0.44,18037.48,18043.71,18075.72,18097.45,17534.87
1235,2023-01-23,18118.45,18162.60,18063.45,18118.55,99.15,-55.00,0.50,18057.75,18053.69,18079.61,18098.28,17540.68
1236,2023-01-24,18183.95,18201.25,18078.65,18118.30,122.60,-105.30,-0.00,18072.89,18062.30,18083.13,18099.07,17546.43
1237,2023-01-25,18093.35,18100.60,17846.15,17891.95,254.45,-247.20,-1.25,18027.66,18039.59,18065.75,18090.95,17549.87


### Simple Candlestick Plot with 14 and 50 day Exponential Moving Averages

In [39]:
# Call the get history function for 1 year
df_nse_1_year = history_func(1)

# Add 14 and 50 Day Exponential Moving Average Columns to dataframe
ema_list = ema_calc(df_nse_1_year['Close'], 14)
df_nse_1_year['14 day EMA'] = ema_list

ema_list = ema_calc(df_nse_1_year['Close'], 50)
df_nse_1_year['50 day EMA'] = ema_list
del ema_list

# Plot the candlestick time series graph for the last year's worth of data with the 14 and 50 day exponential moving averages
candlestick = pgo.Figure(data = [pgo.Candlestick(x=df_nse_1_year['Date'], open=df_nse_1_year['Open'], high=df_nse_1_year['High'], 
low=df_nse_1_year['Low'], close=df_nse_1_year['Close'], line = dict(width = 1), name = 'Candlestick'),
pgo.Scatter(x = df_nse_1_year['Date'], y = df_nse_1_year['14 day EMA'], line = dict(width=1), name = '14 day EMA'),
pgo.Scatter(x = df_nse_1_year['Date'], y = df_nse_1_year['50 day EMA'], line = dict(width=1), name = '50 day EMA')])

candlestick.update_yaxes(fixedrange=False)

candlestick.show()

### Heikin Ashi Candlestick Plot

In [ ]:
# Copying structure of candlestick dataframe into a new dataframe to perform Heikin Ashi calculations
df_heikin_ashi = df_nse_1_year

# First, calculate the Heikin Ashi Close price
df_heikin_ashi['Close'] = round((df_nse_1_year['Close'] + df_nse_1_year['High'] + df_nse_1_year['Low'] + df_nse_1_year['Open']) / 4, 2)

# Next, calculate the Heikin Ashi Open price
for i in df_heikin_ashi.index:
    if(i == 0):
        df_heikin_ashi.at[0, 'Open'] = round((df_nse_1_year['Open'][i] + df_nse_1_year['Close'][i]) / 2, 2)
    else:
        df_heikin_ashi.at[i, 'Open'] = round((df_heikin_ashi.at[i-1, 'Open'] + df_heikin_ashi.at[i-1, 'Close']) / 2, 2)

# Finally, calculate the Heikin Ashi High and Low prices
for i in df_heikin_ashi.index:
    df_heikin_ashi.at[i, 'High'] = max(df_heikin_ashi.at[i, 'Open'], df_heikin_ashi.at[i, 'Close'], df_nse_1_year.at[i, 'High'])
    df_heikin_ashi.at[i, 'Low'] = min(df_heikin_ashi.at[i, 'Open'], df_heikin_ashi.at[i, 'Close'], df_nse_1_year.at[i, 'Low'])

# Plot the Heikin Ashi Candlestick time series graph
heikin_ashi_candlestick = pgo.Figure(data = pgo.Candlestick(x=df_heikin_ashi['Date'], open=df_heikin_ashi['Open'],
high=df_heikin_ashi['High'], low=df_heikin_ashi['Low'], close=df_heikin_ashi['Close']))
heikin_ashi_candlestick.show()
